Import libraries requred for the script

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from numpy.polynomial import polynomial as npp
from scipy.stats import linregress

Read the CSV files

In [6]:
df_pa = pd.read_csv('VMI_Data_BP1_V02_AK6.csv')


Check the columns in the data

In [7]:
df_pa = df_pa.drop('SPLICE_LIMIT_LOW',1)
df_pa = df_pa.drop('SPLICE_LIMIT_HIGH',1)
print(df_pa.columns)


Index(['MACHINE_ID', 'DATE_TIME_STAMP', 'GT_BARCODE', 'TIRE_TYPE',
       'CUT_LENGTH', 'CONV_WAIT_TIME', 'LFT_SPLICE_LENGTH',
       'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 'SPLICE_OK', 'BP1_S8_L',
       'BP1_S7_L', 'BP1_S6_L', 'BP1_S5_L', 'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L',
       'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 'BP1_S3_R', 'BP1_S4_R',
       'BP1_S5_R', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R', 'BP1_PART',
       'BP1_BARCODE', 'PART_LENGTH', 'BP1_BUILD_DATE', 'BP1_MACHINEID'],
      dtype='object')


Sort the data by timestamp

In [8]:
df_pa.DATE_TIME_STAMP = pd.to_datetime(df_pa.DATE_TIME_STAMP, format="%m/%d/%Y %H:%M:%S.%f")


In [9]:
print(df_pa.shape)
df_pa_dupes = df_pa.duplicated(subset=['CUT_LENGTH', 'CONV_WAIT_TIME', 'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 'BP1_S5_L', 
                                      'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L', 'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 
                                      'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R'])
df_pa = df_pa.drop_duplicates(subset=['CUT_LENGTH', 'CONV_WAIT_TIME', 'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 'BP1_S5_L', 
                                      'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L', 'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 
                                      'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R'])
# df_pa = df_pa.drop_duplicates()
print(df_pa.shape)

(123932, 31)
(123098, 31)


In [16]:
# Duplicate rows grouped by a column, typically ID column
# pd.concat(g for _, g in df_pa.groupby("SPLICE_OK") if len(g) > 1)


Check missing data

In [5]:
df_pa['CUT_LENGTH'] = pd.to_numeric(df_pa['CUT_LENGTH'], errors='coerce')
df_pa['CONV_WAIT_TIME'] = pd.to_numeric(df_pa['CONV_WAIT_TIME'], errors='coerce')
df_pa['LFT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['LFT_SPLICE_LENGTH'], errors='coerce')
df_pa['MID_SPLICE_LENGTH'] = pd.to_numeric(df_pa['MID_SPLICE_LENGTH'], errors='coerce')
df_pa['RHT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['RHT_SPLICE_LENGTH'], errors='coerce')


In [6]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))
df_pa = df_pa.dropna()



                   Total   Percent
BP1_MACHINEID       5464  4.408869
BP1_BUILD_DATE      5464  4.408869
BP1_BARCODE         5464  4.408869
BP1_PART            5464  4.408869
RHT_SPLICE_LENGTH    134  0.108124
LFT_SPLICE_LENGTH     49  0.039538
MID_SPLICE_LENGTH     46  0.037117
BP1_S7_L               0  0.000000
BP1_S8_L               0  0.000000
SPLICE_OK              0  0.000000


In [7]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(20))



                   Total  Percent
BP1_MACHINEID          0      0.0
BP1_S4_L               0      0.0
DATE_TIME_STAMP        0      0.0
GT_BARCODE             0      0.0
TIRE_TYPE              0      0.0
CUT_LENGTH             0      0.0
CONV_WAIT_TIME         0      0.0
LFT_SPLICE_LENGTH      0      0.0
MID_SPLICE_LENGTH      0      0.0
RHT_SPLICE_LENGTH      0      0.0
SPLICE_OK              0      0.0
BP1_S8_L               0      0.0
BP1_S7_L               0      0.0
BP1_S6_L               0      0.0
BP1_S5_L               0      0.0
BP1_S3_L               0      0.0
BP1_BUILD_DATE         0      0.0
BP1_S2_L               0      0.0
BP1_S1_ML              0      0.0
BP1_S1_MR              0      0.0


In [8]:
df_pa = df_pa.sort_values('DATE_TIME_STAMP')
df_pa.reset_index(drop=True)

,MACHINE_ID,DATE_TIME_STAMP,GT_BARCODE,TIRE_TYPE,CUT_LENGTH,CONV_WAIT_TIME,LFT_SPLICE_LENGTH,MID_SPLICE_LENGTH,RHT_SPLICE_LENGTH,SPLICE_OK,...,BP1_S4_R,BP1_S5_R,BP1_S6_R,BP1_S7_R,BP1_S8_R,BP1_PART,BP1_BARCODE,PART_LENGTH,BP1_BUILD_DATE,BP1_MACHINEID
0,V02,2018-09-01 00:05:28.500,AK6T27O,AK6,1709.016,48.340,8.021028,5.521104,6.561448,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,1,14:25.2,123CTBPH05
1,V02,2018-09-01 00:10:44.000,AK6T27P,AK6,1708.708,105.367,6.249880,5.013739,5.731807,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,2,14:25.2,123CTBPH05
2,V02,2018-09-01 00:13:30.900,AK6T27Q,AK6,1708.376,314.499,7.917557,5.710968,6.457978,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,3,14:25.2,123CTBPH05
3,V02,2018-09-01 00:14:31.700,AK6T27R,AK6,1709.362,293.230,7.813074,7.258098,7.388051,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,4,14:25.2,123CTBPH05
4,V02,2018-09-01 00:16:52.600,AK6T27S,AK6,1710.018,144.220,8.539536,6.968237,7.496296,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,5,14:25.2,123CTBPH05
5,V02,2018-09-01 00:17:54.500,AK6T27T,AK6,1709.094,42.295,5.422554,5.618785,6.251761,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,6,14:25.2,123CTBPH05
6,V02,2018-09-01 00:18:54.200,AK6T27U,AK6,1709.140,113.722,8.231874,6.914693,8.549954,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,7,14:25.2,123CTBPH05
7,V02,2018-09-01 00:19:52.600,AK6T27V,AK6,1709.544,39.130,6.040480,6.511233,8.856313,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,8,14:25.2,123CTBPH05
8,V02,2018-09-01 00:28:40.500,AK6T27W,AK6,1709.417,36.863,6.255669,5.103172,4.998978,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,9,14:25.2,123CTBPH05
9,V02,2018-09-01 00:29:40.400,AK6T27X,AK6,1708.686,55.367,5.730360,5.417055,6.459715,1,...,0.10,0.01,0.0,0.0,0.0,KALCB78,083018011425KALCB78 08,10,14:25.2,123CTBPH05


Adding ID column for slope calculations
Probably do not need it 

In [9]:

def polyfit(x):
    return npp.polyfit(list(range(len(x))), x, 1)[0]

def linearregress(x):
    slope, intercept, r_value, p_value, std_err = linregress(list(range(len(x))), x)
    return slope


In [10]:
# This code can help write logic for roll length - it uses global var in function

# import pandas as pd
# import numpy as np

# data = np.array([[10, 2, 10, 10],
#                  [10, 3, 60, 100],
#                  [np.nan] * 4,
#                  [10, 22, 280, 250]]).T
# idx = pd.date_range('20150131', end='20150203')
# df = pd.DataFrame(data=data, columns=list('ABCD'), index=idx)
# df
#                A    B     C    D
#  =================================
#  2015-01-31    10   10    NaN  10
#  2015-02-01    2    3     NaN  22 
#  2015-02-02    10   60    NaN  280
#  2015-02-03    10   100   NaN  250

# def calculate(mul, add):
#     global value
#     value = value * mul + add
#     return value

# value = df.loc['2015-01-31', 'D']
# df.loc['2015-01-31', 'C'] = value
# df.loc['2015-02-01':, 'C'] = df.loc['2015-02-01':].apply(lambda row: calculate(*row[['A', 'B']]), axis=1)
# df
#                A    B     C     D
#  =================================
#  2015-01-31    10   10    10    10
#  2015-02-01    2    3     23    22 
#  2015-02-02    10   60    290   280
#  2015-02-03    10   100   3000  250

In [11]:
# df_pa['LFT_SPLICE_DELTA'] = df_pa['LFT_SPLICE_LENGTH'] - (3.5+11.0)/2
# df_pa['LFT_SPLICE_PREV'] = df_pa.LFT_SPLICE_DELTA.shift(1)
# df_pa['LFT_SPLICE_MA5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
# df_pa['LFT_SPLICE_MA10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
# df_pa['LFT_SPLICE_MA20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
# df_pa['LFT_SPLICE_MA50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
# df_pa['LFT_SPLICE_SLOPE5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
# df_pa['LFT_SPLICE_SLOPE10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
# df_pa['LFT_SPLICE_SLOPE20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
# df_pa['LFT_SPLICE_SLOPE50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


# df_pa['MID_SPLICE_DELTA'] = df_pa['MID_SPLICE_LENGTH'] - (4.9+18.0)/2
# df_pa['MID_SPLICE_PREV'] = df_pa.MID_SPLICE_DELTA.shift(1)
# df_pa['MID_SPLICE_MA5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
# df_pa['MID_SPLICE_MA10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
# df_pa['MID_SPLICE_MA20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
# df_pa['MID_SPLICE_MA50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
# df_pa['MID_SPLICE_SLOPE5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
# df_pa['MID_SPLICE_SLOPE10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
# df_pa['MID_SPLICE_SLOPE20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
# df_pa['MID_SPLICE_SLOPE50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


# df_pa['RHT_SPLICE_DELTA'] = df_pa['RHT_SPLICE_LENGTH'] - (3.5+11.0)/2
# df_pa['RHT_SPLICE_PREV'] = df_pa.RHT_SPLICE_DELTA.shift(1)
# df_pa['RHT_SPLICE_MA5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
# df_pa['RHT_SPLICE_MA10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
# df_pa['RHT_SPLICE_MA20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
# df_pa['RHT_SPLICE_MA50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
# df_pa['RHT_SPLICE_SLOPE5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
# df_pa['RHT_SPLICE_SLOPE10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
# df_pa['RHT_SPLICE_SLOPE20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
# df_pa['RHT_SPLICE_SLOPE50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)

# 'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 'BP1_S5_L', 
#                                       'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L', 'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 
#                                       'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R'

df_pa['BP1_S5_L_DELTA'] = df_pa.BP1_S5_L - df_pa.BP1_S8_L.shift(1)
df_pa['BP1_S6_L_MA2'] = df_pa.BP1_S5_L.rolling(window=2,min_periods=1).mean()
df_pa['BP1_S5_L_SLOPE2'] = df_pa.BP1_S5_L.rolling(window=2,min_periods=1).apply(linearregress)
df_pa['BP1_S5_L_MA5'] = df_pa.BP1_S5_L.rolling(window=5,min_periods=1).mean()
df_pa['BP1_S5_L_SLOPE5'] = df_pa.BP1_S5_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S7_L_DELTA'] = df_pa.BP1_S7_L - df_pa.BP1_S7_L.shift(1)
# df_pa['BP1_S7_L_MA2'] = df_pa.BP1_S7_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S7_L_SLOPE2'] = df_pa.BP1_S7_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S7_L_MA5'] = df_pa.BP1_S7_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S7_L_SLOPE5'] = df_pa.BP1_S7_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)

# df_pa['BP1_S8_L_DELTA'] = df_pa.BP1_S8_L - df_pa.BP1_S8_L.shift(1)
# df_pa['BP1_S8_L_MA2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE2'] = df_pa.BP1_S8_L.rolling(window=2,min_periods=1).apply(linearregress)
# df_pa['BP1_S8_L_MA5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).mean()
# df_pa['BP1_S8_L_SLOPE5'] = df_pa.BP1_S8_L.rolling(window=5,min_periods=1).apply(linearregress)



In [12]:
# Delete first 50 rows - so that SMA and Slopes are consistent
df_pa = df_pa.iloc[50:]


In [13]:
# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             df_pa['LFT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             df_pa['MID_SPLICE_LENGTH'].mean(), (18.0 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             18.0,df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5,(3.5 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             df_pa['RHT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['RHT_SPLICE_LENGTH'].max()]

# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#             (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + (18.0 - 4.9)/3),
#             (18.0 - (18.0 - 4.9)/3), 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#              (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

In [14]:
# bin_names=['Bad','OK','Good','OKH','BadH']
bin_names=['0','1','0H']

In [15]:
# def calSpliceGrade(row):
#     if row['LFT_SPLICE_GRADE'] == 'Bad' or row['MID_SPLICE_GRADE'] == 'Bad' or row['RHT_SPLICE_GRADE'] == 'Bad':
#         return 'Bad'
#     if row['LFT_SPLICE_GRADE'] == 'OK' or row['MID_SPLICE_GRADE'] == 'OK' or row['RHT_SPLICE_GRADE'] == 'OK':
#         return 'OK'
#     return 'Good'

def calSpliceGrade(row):
    if (row['LFT_SPLICE_GRADE'] == 0) or (row['MID_SPLICE_GRADE'] == 0) or (row['RHT_SPLICE_GRADE'] == 0):
        return 0
    return 1

In [16]:
# df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

df_pa['LFT_SPLICE_GRADE'] = pd.cut(df_pa['LFT_SPLICE_LENGTH'],lft_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['MID_SPLICE_GRADE'] = pd.cut(df_pa['MID_SPLICE_LENGTH'],mid_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['RHT_SPLICE_GRADE'] = pd.cut(df_pa['RHT_SPLICE_LENGTH'],rht_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['SPLICE_GRADE'] = df_pa.apply(calSpliceGrade, axis=1) 


In [17]:
#df_pa['LFT_SPLICE_GRADE']
# one_hot=pd.get_dummies(df_pa['LFT_SPLICE_GRADE'])
# df_pa = df_pa.drop('LFT_SPLICE_GRADE', axis = 1)
# df_pa = df_pa.join(one_hot)


In [18]:
#df_pa.head
df_pa.to_csv('VMI_Data_BP1_V02_AK6_mod01.csv', header=True, index=False)
